<span style="color:red">Abgegeben von (Name, Vorname):</span> 
Goxhufi, Driton

# WordNet II: Distanz- und Ähnlichkeitsmaße 

Für dieses Notebook gibt es eine begeleitendes Skript auf Ilias: `skript-wordnet.pdf`

Die Basiselemente von [WordNet](https://wordnet.princeton.edu) wurde in der letzten Sitzung bereits vorgestellt. In dieser Sitzung beschäftigen wir uns mit dem (indirekten) Verhältnis zwischen den Synsets in WordNet, wie wir damit die konzeptuelle Ähnlichkeit bemessen und schließlich ein Wort in einem Satz disambiguieren können.

Wir importieren wie gewohnt das WordNet-Modul von NLTK. 

In [1]:
from nltk.corpus import wordnet as wn

Immer griffbereit:
- Website: https://www.nltk.org/
- Buch: https://www.nltk.org/book/
- Module: https://www.nltk.org/py-modindex.html
- Beispiele: http://www.nltk.org/howto/

## Distanzmaße: Minimale Pfadlänge

Die Synsets in WordNet sind über Relationskanten (Hyponomie, Hyperonymie, Meronymie, Antonymie, ...) verbunden, die als Sequenz einen sogenannten **Pfad** bilden. Da wir es hier mit einem Graphen zu tun haben, kann es mehrere solcher Pfade geben (im Unterschied zu Bäumen). 

<img src="Fellbaum(2006)-Figure1-part.png" alt="aus Fellbaum (2006)" style="width: 500px;"/>

Am interessantesten ist sicherlich der kürzeste Pfad zwischen zwei Synsets: 

$$pathlen(s_1 ,s_2 ) = \text{die Anzahl der Kanten im kürzesten Pfad zwischen } s_1 \text{ und }
s_2$$

NLTK stellt hier die Methode `shortest_path_distance()` für Synset-Objekte zur Verfügung. Aber **Vorsicht**: `shortest_path_distance()` betrachtet nur die Hyponymie-Relation.




In [2]:
print("Ergebnisse von shortest_path_distance() bei unterschiedlichen Relationstypen:")
print("gemeinsames direktes Hyponym: {}".format(wn.synset('organism.n.01').shortest_path_distance(wn.synset('causal_agent.n.01'))))
print("Hyponymie: {}".format(wn.synset('bank.n.01').shortest_path_distance(wn.synset('waterside.n.01'))))
print("Meronymie: {}".format(wn.synset('water.n.01').shortest_path_distance(wn.synset('oxygen.n.01'))))
print("Antonymie: {}".format(wn.synset('living.n.02').shortest_path_distance(wn.synset('dead.n.01'))))
print("Derivationally related: {}".format(wn.synset('decision.n.01').shortest_path_distance(wn.synset('decide.v.01'))))

Ergebnisse von shortest_path_distance() bei unterschiedlichen Relationstypen:
gemeinsames direktes Hyponym: 5
Hyponymie: 1
Meronymie: 4
Antonymie: 2
Derivationally related: None


In [3]:
wn.synset('organism.n.01').shortest_path_distance(wn.synset('causal_agent.n.01'))

5

### <span style="color:red">Aufgaben I</span>

<span style="color:red">A1:</span> Schreiben Sie eine Funktion `shortest_hyponomy_pathlength_of_lemmas(lemma1,lemma2)`, die für zwei **Lemmaformen** `lemma1` und `lemma2` den kürzesten Hyponomie-Pfad als ganze Zahl und die entsprechenden Synset-Namen ausgibt! Verwenden Sie als Ausgabeformat eine Liste mit drei Elementen:

    shortest_hyponomy_pathlength_of_lemmas('bank','waterside') -->  [1,'bank.n.01','waterside.n.01']



In [4]:
l_bank = wn.lemmas('bank')[0]
l_waterside = wn.lemmas('waterside')[0]
print(l_bank)
print(l_waterside)

Lemma('bank.n.01.bank')
Lemma('waterside.n.01.waterside')


In [6]:
hyp = lambda s:s.hypernyms()
list(l_bank.synset().closure(hyp))

[Synset('slope.n.01'),
 Synset('geological_formation.n.01'),
 Synset('object.n.01'),
 Synset('physical_entity.n.01'),
 Synset('entity.n.01')]

In [7]:
l_bank.synset().shortest_path_distance(l_waterside.synset())

1

In [10]:
# Lösung A1
def shortest_hyponomy_pathlength_of_lemmas(lemma1, lemma2):
    shpl = []
    
    l1 = wn.lemmas(lemma1)[0]
    l2 = wn.lemmas(lemma2)[0]
    l1_synset = l1.synset()
    l2_synset = l2.synset()
    
    d =l1_synset.shortest_path_distance(l2_synset)
    shpl.append(d)
    shpl.append(l1_synset.name())
    shpl.append(l2_synset.name())
    
    return shpl

In [204]:
# Test für A1

["{:10}, {:10}: {}".format(lemma1,lemma2,shortest_hyponomy_pathlength_of_lemmas(lemma1,lemma2))
    for  lemma1,lemma2 
    in [['bank','waterside'],
        ['person','cause'],
        ['person','organism'],
        ['cause','organism']]]


["bank      , waterside : [1, 'bank.n.01', 'waterside.n.01']",
 "person    , cause     : [10, 'person.n.01', 'cause.n.01']",
 "person    , organism  : [1, 'person.n.01', 'organism.n.01']",
 "cause     , organism  : [12, 'cause.n.01', 'organism.n.01']"]

## Ähnlichkeitsmaße

Es gibt eine Reihe von Größen, die die semantische Ähnlichkeit oder semantische Nähe von zwei Synsets bemessen soll. 

### Pfadbasiert

Die Pfadlänge an sich ist ein wichtiger und einfacher Indikator für die semantische Ähnlichkeit oder semantische Nähe von zwei Synsets: Je kleiner die (kleinste) Pfadlänge, desto ähnlicher sind die Synsets und die enthaltenen Lexeme/Lemmata.

$$sim_{path}(s_1,s_2) = \frac{1}{pathlen(s_1,s_2)+1}$$

NLTK stellt dafür die Methode `path_similarity()` zur Verfügung.

### Pfadbasiert + LCS

Eine Variante der pfadbasierten Ähnlichkeit, die die Tiefe der Synsets berücksichtigt ist die **Wu-Palmer Similarity** (`wup-similarity()`). Die Idee ist hier, dass sich die Einbettungstiefe $depth$ des "lowest common subsumer" $lcs$ (niedrigstes gemeinsames Hypernym) positiv auf das Ähnlichkeitsmaß auswirkt. 

$$sim_{wup}(s_1,s_2) = \frac{2 * depth(lcs(s_1,s_2))}{pathlen(s_1,lcs(s_1,s_2))+pathlen(s_2,lcs(s_1,s_2))}$$

### Wahrscheinlichkeiten der Synsets und des LCS

Es gibt aber auch Ähnlichkeitsmaße, die Pfade außer Acht lassen und nur die Wahrscheinlichkeit der Synsets und deren LCS betrachten. Die benötigten Wahrscheinlichkeiten können z.B. mit Wort-Frequenzen in Corpora abgeschätzt werden:

$$P(s) = \frac{\sum_{w \in lemmas(s)} count(w)}{N}$$

Die Wahrscheinlichkeit der Synsets nimmt damit tendenziell ab, je spezifischer sie sind: 

<img src="Jurafsky,Martin(2018)-FigureC.6.PNG" alt="aus Jurafsky & Martin (2018)" style="width: 220px;"/>

Ein bekanntes Ähnlichkeitsmaß ist hier die **Resnik Similarity** (`res-similarity()`), die nur die Wahrscheinlichkeit (oder genauer gesagt den Informationsgehalt) des LCS betrachtet:

$$sim_{Resnik}(s_1,s_2) = -\log P(lcs(s_1,s_2))$$

Weitentwicklungen der Resnik Similarity berücksichtigen auch die Wahrscheinlichkeit der Synsets (und damit indirekt die Pfadlänge zum LCS).

NLTK enthält davon:
- **Lin Similarity** (`lin-similarity()`): $sim_{Lin}(s_1,s_2) = \frac{2 * \log P(lcs(s_1,s_2))}{\log P(s_1) + P(s_2)}$
- **Jiang-Conrath Distance** (`jcn_similarity()`): $sim_{JC}(s_1,s_2) = 2 * \log P(lcs(s_1,s_2)) - (\log P(s_1) + P(s_2))$ 

In [15]:
# Die wahrscheinlichkeitsbezogenen Maße benötigen zusätzliche Daten, 
# die zuvor importiert werden müssen.  
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')
semcor_ic = wordnet_ic.ic('ic-semcor.dat')

def compare_similarity_measures(s1,s2) :
    print("Synsets: {}, {}".format(s1.name(),s2.name()))
    print("path_sim: {}".format(s1.path_similarity(s2)))
    print("wup_sim: {}".format(s1.wup_similarity(s2)))
    print("res_sim: {}".format(s1.res_similarity(s2,brown_ic)))
    print("lin_sim: {}".format(s1.lin_similarity(s2,brown_ic)))
    print("jcn_sim: {}".format(s1.jcn_similarity(s2,brown_ic)))

compare_similarity_measures(wn.synset('bank.n.1'),wn.synset('sea.n.1'))
print("")
compare_similarity_measures(wn.synset('bank.n.1'),wn.synset('bank.n.2'))
print("")
compare_similarity_measures(wn.synset('organism.n.01'),wn.synset('causal_agent.n.01'))

Synsets: bank.n.01, sea.n.01
path_sim: 0.125
wup_sim: 0.36363636363636365
res_sim: 0.8017591149538994
lin_sim: 0.0837782004484866
jcn_sim: 0.05702384600192556

Synsets: bank.n.01, depository_financial_institution.n.01
path_sim: 0.07692307692307693
wup_sim: 0.14285714285714285
res_sim: -0.0
lin_sim: -0.0
jcn_sim: 0.05165835201605558

Synsets: organism.n.01, causal_agent.n.01
path_sim: 0.16666666666666666
wup_sim: 0.4444444444444444
res_sim: 0.8017591149538994
lin_sim: 0.3557998922959274
jcn_sim: 0.3444380465758159


### <span style="color:red">Aufgaben II</span>

Wir können nun Aufgabe A1 so erweitern, dass $lemmasim$ für zwei Lemmaformen ($l_1, l_2$) und eines der genannten Ähnlichkeitsmaße ($sim$) berechnet wird.

$$lemmasim(l_1,l_2) = \max_{s_1 \in synsets(l_1)\\s_2 \in synsets(l_2)} sim(s_1,s_2)$$

<span style="color:red">A2:</span> Schreiben Sie eine Funktion `lemmasim(l1,l2,sim)`, die bezogen auf ein Ähnlichkeitsmaß `sim` (path, wup, res, lin, jcn) den höchsten Wert und die dazugehörigen Synsets ausgibt!  



In [223]:
# Lösung A2
def lemmasim(l1, l2, sim):
    shlp = []
    best = 0
    
    l1 = wn.lemmas(l1)[0]
    l2 = wn.lemmas(l2)[0]
    l1_synset = l1.synset()
    l2_synset = l2.synset()
    
    # s1.name(),s2.name()
    if sim == "path":
        best = l1_synset.path_similarity(l2_synset)
    elif sim == "wup":
        best = l1_synset.wup_similarity(l2_synset)
    elif sim == "res":
        best = l1_synset.res_similarity(l2_synset,brown_ic)
    elif sim == "lin":
        best = l1_synset.lin_similarity(l2_synset,brown_ic)
    elif sim == "jcn":
        best = l1_synset.jcn_similarity(l2_synset,brown_ic)
    else:
        print("Your input was not acceped. Please use one of these similarity measure: path, wup, res, lin, jcn")
    shlp.append(best)
    shlp.append(l1_synset.name())
    shlp.append(l2_synset.name())
    
    return shlp

In [224]:
l_bank.synset().wup_similarity(l_waterside.synset())

0.9230769230769231

In [225]:
# Tests für A2

["{:10}, {:10}: {}".format(lemma1,lemma2,lemmasim(lemma1,lemma2,"wup"))
    for  lemma1,lemma2 
    in [['bank','waterside'],
        ['person','cause'],
        ['person','organism'],
        ['cause','organism']]]

["bank      , waterside : [0.9230769230769231, 'bank.n.01', 'waterside.n.01']",
 "person    , cause     : [0.16666666666666666, 'person.n.01', 'cause.n.01']",
 "person    , organism  : [0.9230769230769231, 'person.n.01', 'organism.n.01']",
 "cause     , organism  : [0.14285714285714285, 'cause.n.01', 'organism.n.01']"]

## Exkursion: SemCor

SemCor ist ein mit WordNet-Synsets annotierter Teil des Brown-Corpus (u.a.) und umfasst ca. 240 000 Worttoken.

SemCor ist in NLTK enthalten und als `corpus.reader`-Modul abrufbar: https://www.nltk.org/api/nltk.corpus.reader.html#module-nltk.corpus.reader.semcor 


In [44]:
from nltk.corpus import semcor

list(map(str, semcor.tagged_chunks(tag='both')[:3]))

['(DT The)',
 "(Lemma('group.n.01.group') (NE (NNP Fulton County Grand Jury)))",
 "(Lemma('state.v.01.say') (VB said))"]

## Anwendung: Disambiguierung von *interest*

Wie können wir nun mit den oben behandelten Ähnlichkeitsmaßen ein Token disambiguieren? Und wie unterscheiden sich die Ähnlichkeitsmaße in ihrer "Treffsicherheit"?

Wir werden uns hier auf ein Nomen konzentrieren, nämlich *interest* mit seinen 7 Bedeutungen:

In [46]:
[[syns.name(),syns.definition()] for syns in wn.synsets('interest',pos='n')]

[['interest.n.01',
  'a sense of concern with and curiosity about someone or something'],
 ['sake.n.01', 'a reason for wanting something done'],
 ['interest.n.03',
  "the power of attracting or holding one's attention (because it is unusual or exciting etc.)"],
 ['interest.n.04',
  'a fixed charge for borrowing money; usually a percentage of the amount borrowed'],
 ['interest.n.05',
  '(law) a right or legal share of something; a financial involvement with something'],
 ['interest.n.06',
  '(usually plural) a social group whose members control some field of activity and who have common aims'],
 ['pastime.n.01',
  "a diversion that occupies one's time and thoughts (usually pleasantly)"]]

Warum *interest*? Weil dafür Testdaten aus einem [Senseval-Wettbewerb](https://en.wikipedia.org/wiki/SemEval) in NLTK vorliegen.

### Exkurs: Vorbereitung Senseval-Testdaten

Die Senseval-Testdaten sind Teil von NLTK (https://www.nltk.org/api/nltk.corpus.reader.html#module-nltk.corpus.reader.senseval), müssen aber zunächst vorbereitet werden. 

Es gibt unterschiedliche "Dateien" für unterschiedliche Lemmaformen, die bei dem Senseval-Wettbewerb disambiguiert werden sollten ("target words"):

In [47]:
from nltk.corpus import senseval
senseval.fileids()

['hard.pos', 'interest.pos', 'line.pos', 'serve.pos']

Davon interessiert uns aber nur `interest.pos`:

In [48]:
from nltk.corpus import senseval
senseval.instances('interest.pos')

[SensevalInstance(word='interest-n', position=18, context=[('yields', 'NNS'), ('on', 'IN'), ('money-market', 'JJ'), ('mutual', 'JJ'), ('funds', 'NNS'), ('continued', 'VBD'), ('to', 'TO'), ('slide', 'VB'), (',', ','), ('amid', 'IN'), ('signs', 'VBZ'), ('that', 'IN'), ('portfolio', 'NN'), ('managers', 'NNS'), ('expect', 'VBP'), ('further', 'JJ'), ('declines', 'NNS'), ('in', 'IN'), ('interest', 'NN'), ('rates', 'NNS'), ('.', '.')], senses=('interest_6',)), SensevalInstance(word='interest-n', position=7, context=[('longer', 'RB'), ('maturities', 'NNS'), ('are', 'VBP'), ('thought', 'VBN'), ('to', 'TO'), ('indicate', 'VB'), ('declining', 'VBG'), ('interest', 'NN'), ('rates', 'NNS'), ('because', 'IN'), ('they', 'PRP'), ('permit', 'VBP'), ('portfolio', 'NN'), ('managers', 'NNS'), ('to', 'TO'), ('retain', 'VB'), ('relatively', 'RB'), ('higher', 'JJR'), ('rates', 'NNS'), ('for', 'IN'), ('a', 'DT'), ('longer', 'RB'), ('period', 'NN'), ('.', '.')], senses=('interest_6',)), ...]

Die Attribute dieser Instanzen können wir ganz leicht einzeln ausgeben: `senseval.instances('interest.pos')[0].word` etc.

Wir stellen aber fest, dass die Synset-Namen nicht stimmen (z.B. `interest_6`). Es handelt sich dabei um alte WordNet-Bezeichnungen, die wir mit `SV_SENSE_MAP` übersetzen werden: 

In [49]:
# Copied from https://stackoverflow.com/a/16391584/6452961

# A map of SENSEVAL senses to WordNet 3.0 senses.
# SENSEVAL-2 uses WordNet 1.7, which is no longer installable on most modern
# machines and is not the version that the NLTK comes with.
# As a consequence, we have to manually map the following
# senses to their equivalent(s).
SV_SENSE_MAP = {
    "HARD1": ["difficult.a.01"],    # not easy, requiring great physical or mental
    "HARD2": ["hard.a.02",          # dispassionate
              "difficult.a.01"],
    "HARD3": ["hard.a.03"],         # resisting weight or pressure
    "interest_1": ["interest.n.01"], # readiness to give attention
    "interest_2": ["interest.n.03"], # quality of causing attention to be given to
    "interest_3": ["pastime.n.01"],  # activity, etc. that one gives attention to
    "interest_4": ["sake.n.01"],     # advantage, advancement or favor
    "interest_5": ["interest.n.05"], # a share in a company or business
    "interest_6": ["interest.n.04"], # money paid for the use of money
    "cord": ["line.n.18"],          # something (as a cord or rope) that is long and thin and flexible
    "formation": ["line.n.01","line.n.03"], # a formation of people or things one beside another
    "text": ["line.n.05"],                 # text consisting of a row of words written across a page or computer screen
    "phone": ["telephone_line.n.02"],   # a telephone connection
    "product": ["line.n.22"],       # a particular kind of product or merchandise
    "division": ["line.n.29"],      # a conceptual separation or distinction
    "SERVE12": ["serve.v.02"],       # do duty or hold offices; serve in a specific function
    "SERVE10": ["serve.v.06"], # provide (usually but not necessarily food)
    "SERVE2": ["serve.v.01"],       # serve a purpose, role, or function
    "SERVE6": ["service.v.01"]      # be used by; as of a utility
}

Jetzt können wir die Umwandlung der Senseval-Daten für *interest* vornehmen:

In [50]:
from nltk.corpus import senseval
interestGoldData = [[SV_SENSE_MAP[inst.senses[0]][0],inst.position,inst.context] 
                        for inst in senseval.instances('interest.pos')]
interestTestData = [['', inst[1], inst[2]] for inst in interestGoldData]

print(interestGoldData[0])
print(interestTestData[0])

['interest.n.04', 18, [('yields', 'NNS'), ('on', 'IN'), ('money-market', 'JJ'), ('mutual', 'JJ'), ('funds', 'NNS'), ('continued', 'VBD'), ('to', 'TO'), ('slide', 'VB'), (',', ','), ('amid', 'IN'), ('signs', 'VBZ'), ('that', 'IN'), ('portfolio', 'NN'), ('managers', 'NNS'), ('expect', 'VBP'), ('further', 'JJ'), ('declines', 'NNS'), ('in', 'IN'), ('interest', 'NN'), ('rates', 'NNS'), ('.', '.')]]
['', 18, [('yields', 'NNS'), ('on', 'IN'), ('money-market', 'JJ'), ('mutual', 'JJ'), ('funds', 'NNS'), ('continued', 'VBD'), ('to', 'TO'), ('slide', 'VB'), (',', ','), ('amid', 'IN'), ('signs', 'VBZ'), ('that', 'IN'), ('portfolio', 'NN'), ('managers', 'NNS'), ('expect', 'VBP'), ('further', 'JJ'), ('declines', 'NNS'), ('in', 'IN'), ('interest', 'NN'), ('rates', 'NNS'), ('.', '.')]]


### <span style="color:red">Aufgaben III</span>

Mit den Testdaten in `interestTestData` und den Golddaten in `interestGoldData` können wir verschiedene Ansätze und Ähnlichkeitsmaße für die Disambiguierung von *interest* evaluieren. 

<span style="color:red">A3:</span> Vervollständigen Sie die Funktion `disambiguate_interest`, indem Sie mindestens eine der oben erwähnten Ähnlichkeitsmetriken verwenden!

In [65]:
x = interestTestData[0]
x

['',
 18,
 [('yields', 'NNS'),
  ('on', 'IN'),
  ('money-market', 'JJ'),
  ('mutual', 'JJ'),
  ('funds', 'NNS'),
  ('continued', 'VBD'),
  ('to', 'TO'),
  ('slide', 'VB'),
  (',', ','),
  ('amid', 'IN'),
  ('signs', 'VBZ'),
  ('that', 'IN'),
  ('portfolio', 'NN'),
  ('managers', 'NNS'),
  ('expect', 'VBP'),
  ('further', 'JJ'),
  ('declines', 'NNS'),
  ('in', 'IN'),
  ('interest', 'NN'),
  ('rates', 'NNS'),
  ('.', '.')]]

In [117]:
interestTestData[0][0]

''

In [118]:
interestGoldData[0][0]

'interest.n.04'

In [104]:
senseval.instances('interest.pos')[0].senses

('interest_6',)

In [140]:
o = interestTestData[0][0]
p = interestTestData[0][1]
c = interestTestData[0][2]
print(o,p,c)

 18 [('yields', 'NNS'), ('on', 'IN'), ('money-market', 'JJ'), ('mutual', 'JJ'), ('funds', 'NNS'), ('continued', 'VBD'), ('to', 'TO'), ('slide', 'VB'), (',', ','), ('amid', 'IN'), ('signs', 'VBZ'), ('that', 'IN'), ('portfolio', 'NN'), ('managers', 'NNS'), ('expect', 'VBP'), ('further', 'JJ'), ('declines', 'NNS'), ('in', 'IN'), ('interest', 'NN'), ('rates', 'NNS'), ('.', '.')]


In [141]:
c

[('yields', 'NNS'),
 ('on', 'IN'),
 ('money-market', 'JJ'),
 ('mutual', 'JJ'),
 ('funds', 'NNS'),
 ('continued', 'VBD'),
 ('to', 'TO'),
 ('slide', 'VB'),
 (',', ','),
 ('amid', 'IN'),
 ('signs', 'VBZ'),
 ('that', 'IN'),
 ('portfolio', 'NN'),
 ('managers', 'NNS'),
 ('expect', 'VBP'),
 ('further', 'JJ'),
 ('declines', 'NNS'),
 ('in', 'IN'),
 ('interest', 'NN'),
 ('rates', 'NNS'),
 ('.', '.')]

In [193]:
# lemmasim(c[0], interestGoldData[0][2], "res")
g = interestGoldData[0][2]
wn.lemmas(g[2][0])

[]

In [221]:
test_l1 = g[1][0]
test_l2 = c[0][0]
print(test_l1, test_l2)
print(wn.lemmas(test_l1), wn.lemmas(test_l2))

on yields
[Lemma('on.a.01.on'), Lemma('on.a.02.on'), Lemma('along.r.01.on'), Lemma('on.r.02.on'), Lemma('on.r.03.on')] []


In [183]:
def disambiguate_interest(inst) :
    outsyns = inst[0]
    position = inst[1]
    context = inst[2]   # [('yields', 'NNS'), ('on', 'IN'), ('money-market', 'JJ'), ...]
    
    # Lösung A3 - kein plan...
    
#     p_dist = 9999
#     for p in range(len(interestGoldData)):
#         act_dist = abs(position - p)
#         if  act_dist < p_dists:
#             p_dist = act_dist
#             outsyns = interestGoldData[i][0]
#     for k in range(len(interestGoldData[]))
#         for i in range(len(context)):
#             lemmasim(context[i][0], interestGoldData[]
    return outsyns   

In [184]:
# Test für A3

sumTrueDisambiguations = 0
sumFalseDisambiguations = 0

for i in range(len(interestTestData)) :
    if disambiguate_interest(interestTestData[i]) == interestGoldData[i][0] :
        sumTrueDisambiguations += 1
    else :
        sumFalseDisambiguations += 1

accuracyDisambiguations = sumTrueDisambiguations /(sumTrueDisambiguations + sumFalseDisambiguations)

print("Accuracy: {}".format(accuracyDisambiguations))

Accuracy: 0.0
